In [1]:
import CsemigroupsCpp
from CsemigroupsCpp import *
import PyNormaliz
from PyNormaliz import *
import itertools
from scipy.spatial import ConvexHull
# Esto es solo para ver si lo que hago es útil o no
import time

In [2]:
# Calculo el menor cuboide que contenga al diamante.
# INPUT:
#   - d: Vértices del diamante.
# OUTPUT:
#   - Máximos en cada coordenada del cubo.
def Cube(d):
    dim = len(d[0])
    aux = []
    for i in range(dim):
        aux.append(sorted(d, key = lambda x: x[i])[-1][i])
    return(aux)

In [3]:
class Csemigroup:
    def __init__(self,generators):
        self.gaps = None
        self.pf = None
        aux = self.__ComputeMinimalGenerators(generators)
        self.generators = list([list(x) for x in aux])
        self.generators.sort(key=lambda row: row[1:])
        coneSg = Cone(cone=self.generators)
        self.rays = coneSg.ExtremeRays()
        self.hyperplanes = coneSg.SupportHyperplanes()
        if not self.__IsCSemigroup():
            raise Exception("The set do not form a C-Semigroup")
        
    def __IsCSemigroup(self):
        if not axisAreSemigroup(self.generators,self.rays):
            raise Exception("This set does not generate a numerical semigroup.")
        diamondA = diamond(self.rays)
        hull = ConvexHull(diamondA)
        eqDiamond = [list(x) for x in hull.equations]
        boundDiamond = Cube(diamondA)
        it = itertools.product(*[range(i+1) for i in boundDiamond])
        candidates = [list(x) for x in it]
        integerDiamond = filterPoints(candidates,eqDiamond)
        return studyRays(self.rays,self.hyperplanes,integerDiamond,self.generators);

        
    def __ComputeMinimalGenerators(self,generators):
        return computeMSG(generators)
        
    def GetGenerators(self):
        return self.generators
    def GetRays(self):
        return self.rays
    def GetHyperplanes(self):
        return self.hyperplanes
    def GetGaps(self):
        if self.gaps != None:
            return self.gaps
        else:
            multiplicitiesInAxes = multiplicityAllAxes(self.generators,self.rays)
            diamondMult = diamond(multiplicitiesInAxes)
            hull = ConvexHull(diamondMult)
            eqDiamond = [list(x) for x in hull.equations]
            boundDiamond = Cube(diamondMult)
            it = itertools.product(*[range(i+1) for i in boundDiamond])
            candidates = [list(x) for x in it]
            integerDiamond = filterPoints(candidates,eqDiamond)
            #print(len(integerDiamond))
            print(integerDiamond)
            # self.gaps = computeGaps(self.generators,self.rays,self.hyperplanes)
            computeXDiamond(self.generators, self.rays, self.hyperplanes, integerDiamond)

In [6]:
start = time.process_time()
generators = [[3, 1], [7, 1], [8, 2], [9, 1], [9, 2], [10, 1], [10, 3], [10, 4], [11, 2], [13, 1], [14, 5], [15, 6], [17, 7], [19, 8], [21, 2], [24, 2], [25, 2], [26, 11], [27, 2], [28, 2], [29, 2], [44, 3], [45, 3], [61, 4], [62, 4], [76, 5], [77, 5], [93, 6], [108, 7], [43, 3], [59, 4], [60, 4], [92, 6], [42, 18], [56, 24], [63, 27], [70, 30]]
cs = Csemigroup(generators)
time.process_time()-start

8.8879613

In [11]:
cs.GetGaps()

((0, 0), (3, 1), (4, 1), (5, 1), (5, 2), (6, 1), (6, 2), (7, 1), (7, 2), (7, 3), (8, 1), (8, 2), (8, 3), (9, 1), (9, 2), (9, 3), (10, 1), (10, 2), (10, 3), (10, 4), (11, 1), (11, 2), (11, 3), (11, 4), (12, 1), (12, 2), (12, 3), (12, 4), (12, 5), (13, 1), (13, 2), (13, 3), (13, 4), (13, 5), (14, 1), (14, 2), (14, 3), (14, 4), (14, 5), (14, 6), (15, 1), (15, 2), (15, 3), (15, 4), (15, 5), (15, 6), (16, 2), (16, 3), (16, 4), (16, 5), (16, 6), (17, 2), (17, 3), (17, 4), (17, 5), (17, 6), (17, 7), (18, 2), (18, 3), (18, 4), (18, 5), (18, 6), (18, 7), (19, 2), (19, 3), (19, 4), (19, 5), (19, 6), (19, 7), (19, 8), (20, 2), (20, 3), (20, 4), (20, 5), (20, 6), (20, 7), (20, 8), (21, 2), (21, 3), (21, 4), (21, 5), (21, 6), (21, 7), (21, 8), (21, 9), (22, 2), (22, 3), (22, 4), (22, 5), (22, 6), (22, 7), (22, 8), (22, 9), (23, 2), (23, 3), (23, 4), (23, 5), (23, 6), (23, 7), (23, 8), (23, 9), (24, 2), (24, 3), (24, 4), (24, 5), (24, 6), (24, 7), (24, 8), (24, 9), (24, 10), (25, 2), (25, 3), (25, 4

In [5]:
cs.GetRays()

NameError: name 'cs' is not defined

In [7]:
cs.GetHyperplanes()

[[-2, 31], [3, -7]]

In [ ]:
-2*3+31*1

In [ ]:
type([[-0.39391929857916763, 0.9191450300180578, -0.0], [-0.06438227799796505, 0.9979253089684582, -2.543099980919619], [0.06438227799796505, -0.9979253089684582, -0.0], [0.39391929857916763, -0.9191450300180578, -10.373208195918082]][0][0])

In [ ]:
pointBelongsDiamond([0,0],[[-0.39391929857916763, 0.9191450300180578, -0.0], [-0.06438227799796505, 0.9979253089684582, -2.543099980919619], [0.06438227799796505, -0.9979253089684582, -0.0], [0.39391929857916763, -0.9191450300180578, -10.373208195918082]])

In [ ]:
axisAreSemigroup(cs.GetGenerators(),cs.rays)

In [ ]:
studyRays([[7, 3], [31, 2]],[[-2, 31], [3, -7]],((0, 0), (3, 1), (4, 1), (5, 1), (5, 2), (6, 1), (6, 2), (7, 1), (7, 2), (7, 3), (8, 1), (8, 2), (8, 3), (9, 1), (9, 2), (9, 3), (10, 1), (10, 2), (10, 3), (11, 1), (11, 2), (11, 3), (12, 1), (12, 2), (12, 3), (13, 1), (13, 2), (13, 3), (14, 1), (14, 2), (14, 3), (15, 1), (15, 2), (15, 3), (16, 2), (16, 3), (17, 2), (17, 3), (18, 2), (18, 3), (19, 2), (19, 3), (20, 2), (20, 3), (21, 2), (21, 3), (22, 2), (22, 3), (23, 2), (23, 3), (23, 4), (24, 2), (24, 3), (24, 4), (25, 2), (25, 3), (25, 4), (26, 2), (26, 3), (26, 4), (27, 2), (27, 3), (27, 4), (28, 2), (28, 3), (28, 4), (29, 2), (29, 3), (29, 4), (30, 2), (30, 3), (30, 4), (31, 2), (31, 3), (31, 4), (32, 3), (32, 4), (33, 3), (33, 4), (34, 4), (35, 4), (38, 5)),[[3, 1], [7, 1], [9, 1], [10, 1], [13, 1], [8, 2], [9, 2], [11, 2], [21, 2], [24, 2], [25, 2], [27, 2], [28, 2], [29, 2], [10, 3], [44, 3], [45, 3], [43, 3], [10, 4], [61, 4], [62, 4], [59, 4], [60, 4], [14, 5], [76, 5], [77, 5], [15, 6], [93, 6], [92, 6], [17, 7], [108, 7], [19, 8], [26, 11], [42, 18], [56, 24], [63, 27], [70, 30]]);